<a href="https://colab.research.google.com/github/Feru34/Deep_Learning/blob/main/Taller_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip -q
!pip install keras-tuner -q
!pip install kaggle -q
!pip install scikeras -q
# El '-q' hace referencia a 'quiet'. Por lo tanto el output de los paquetes no se muestra completo

  Preparing metadata (setup.py) ... done


In [33]:
import numpy as np
import pandas as pd
from google.colab import files

from pandas_profiling import ProfileReport
from numpy import sqrt

import matplotlib.pyplot as plt
plt.style.use("seaborn-deep")
import seaborn as sns

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Input
from keras.utils import plot_model
import keras_tuner as kt

from scikeras.wrappers import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report

<ipython-input-33-55e46eb7280f>:5: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport
<ipython-input-33-55e46eb7280f>:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-deep")


In [34]:
files.upload() # Cargamos el Kaggle.json (API)

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list #Correr este comando verifica que está bien montado el driver de kaggle

Saving kaggle.json to kaggle.json
ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                          Student Performance Factors                          94KB  2024-09-02 10:53:57           5836        126  1.0              
hanaksoy/customer-purchasing-behaviors                           Customer Purchasing Behaviors                         1KB  2024-09-01 22:18:07           3508         51  1.0              
haseebindata/student-performance-predictions                     Student Performance Predictions                       9KB  2024-08-17 06:57:57          11562        247  0.9411765        
computingvictor/2024-

In [35]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.9/107.7 GB disk)


In [36]:
import pandas as pd
import numpy as np

import os
import os.path as osp
import random
import re
import shutil

#Visualización de datos
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from IPython.display import display, Image

from numpy import asarray

from PIL import Image

from ultralytics import YOLO

In [37]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [38]:
my_seed = 19 #Semilla para reproducibilidad
IMG_EXT = ['.jpg', '.jpeg', '.png', '.bmp'] #Extensiones válidas de imagen
TEXT_EXT = '.txt'
YOLO_VERSION = 'yolov8'

In [39]:
random.seed(my_seed)

In [41]:
!kaggle datasets list

ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                          Student Performance Factors                          94KB  2024-09-02 10:53:57           5836        126  1.0              
hanaksoy/customer-purchasing-behaviors                           Customer Purchasing Behaviors                         1KB  2024-09-01 22:18:07           3508         51  1.0              
haseebindata/student-performance-predictions                     Student Performance Predictions                       9KB  2024-08-17 06:57:57          11562        247  0.9411765        
computingvictor/2024-academic-ranking-of-world-universi

In [42]:
!kaggle datasets download hasibzunair/rsud20k-bangladesh-road-scene-understanding

Dataset URL: https://www.kaggle.com/datasets/hasibzunair/rsud20k-bangladesh-road-scene-understanding
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
100% 10.2G/10.2G [02:40<00:00, 97.9MB/s]
100% 10.2G/10.2G [02:40<00:00, 68.5MB/s]


In [44]:
ROOT_DIR = '/content'
DATASET_NAME = 'rsud20k-bangladesh-road-scene-understanding'
SUB_DATASET_NAME = 'rsud20k-bangladesh-road-scene-understanding.v1-yolov8'

In [45]:
print(f"!unzip {DATASET_NAME}.zip -d {ROOT_DIR}/{DATASET_NAME}")

!unzip rsud20k-bangladesh-road-scene-understanding.zip -d /content/rsud20k-bangladesh-road-scene-understanding


In [46]:
%cd {ROOT_DIR}
!mkdir /{DATASET_NAME}
!unzip {DATASET_NAME}.zip -d {ROOT_DIR}/{DATASET_NAME}

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/rsud20k-bangladesh-road-scene-understanding/rsud20k/labels/train/train6370.txt  
  inflating: /content/rsud20k-bangladesh-road-scene-understanding/rsud20k/labels/train/train6371.txt  
  inflating: /content/rsud20k-bangladesh-road-scene-understanding/rsud20k/labels/train/train6372.txt  
  inflating: /content/rsud20k-bangladesh-road-scene-understanding/rsud20k/labels/train/train6373.txt  
  inflating: /content/rsud20k-bangladesh-road-scene-understanding/rsud20k/labels/train/train6374.txt  
  inflating: /content/rsud20k-bangladesh-road-scene-understanding/rsud20k/labels/train/train6375.txt  
  inflating: /content/rsud20k-bangladesh-road-scene-understanding/rsud20k/labels/train/train6376.txt  
  inflating: /content/rsud20k-bangladesh-road-scene-understanding/rsud20k/labels/train/train6377.txt  
  inflating: /content/rsud20k-bangladesh-road-scene-understanding/rsud20k/labels/train/train6378.txt  
  inflating:

In [47]:
DATA_DIR = f"{ROOT_DIR}/{DATASET_NAME}/{SUB_DATASET_NAME}"
print(DATA_DIR)

/content/rsud20k-bangladesh-road-scene-understanding/rsud20k-bangladesh-road-scene-understanding.v1-yolov8


In [48]:
subdirs = ['train', 'test', 'valid']

In [49]:
train_folder = osp.join(DATA_DIR, subdirs[0])
train_img_folder = osp.join(train_folder, "images")
train_lbl_folder = osp.join(train_folder, "labels")
val_folder = osp.join(DATA_DIR, subdirs[2])
val_img_folder = osp.join(val_folder, "images")
val_lbl_folder = osp.join(val_folder, "images")
test_folder = osp.join(DATA_DIR, subdirs[1])
test_img_folder = osp.join(test_folder, "images")
test_lbl_folder = osp.join(test_folder, "images")

In [50]:
folder_list = [train_img_folder, train_lbl_folder,
                val_img_folder, val_lbl_folder,
                test_img_folder, test_lbl_folder
]

print("Imprimimos las rutas de las carpetas:")
for fa in folder_list:
  print(fa)

Imprimimos las rutas de las carpetas:
/content/rsud20k-bangladesh-road-scene-understanding/rsud20k-bangladesh-road-scene-understanding.v1-yolov8/train/images
/content/rsud20k-bangladesh-road-scene-understanding/rsud20k-bangladesh-road-scene-understanding.v1-yolov8/train/labels
/content/rsud20k-bangladesh-road-scene-understanding/rsud20k-bangladesh-road-scene-understanding.v1-yolov8/valid/images
/content/rsud20k-bangladesh-road-scene-understanding/rsud20k-bangladesh-road-scene-understanding.v1-yolov8/valid/images
/content/rsud20k-bangladesh-road-scene-understanding/rsud20k-bangladesh-road-scene-understanding.v1-yolov8/test/images
/content/rsud20k-bangladesh-road-scene-understanding/rsud20k-bangladesh-road-scene-understanding.v1-yolov8/test/images


In [53]:
folder_list = [train_img_folder, val_img_folder,
               test_img_folder]

In [52]:
img_name_list = []
img_width_list = []
img_height_list = []
img_folder_list = []
img_format_list = []

for fa in folder_list:
  parent = os.path.dirname(fa)
  parent_name = os.path.basename(parent)
  for fname in os.listdir(fa):
    img_folder_list.append(parent_name)
    img_name_list.append(fname)
    img_path = osp.join(fa, fname)
    img_format_list = "ok" if osp.splitext(fname)[-1] in IMG_EXT else "not ok"

    with Image.open(img_path) as img:
      width, height = img.size
      img_width_list.append(width)
      img_height_list.append(height)

data_model = {"folder": img_folder_list, "image_name": img_name_list,
              "width": img_width_list, "height": img_height_list
              , "format": img_format_list}
report_df = pd.DataFrame(data=data_model)

FileNotFoundError: [Errno 2] No such file or directory: '/content/rsud20k-bangladesh-road-scene-understanding/rsud20k-bangladesh-road-scene-understanding.v1-yolov8/train/images'